### 1. Load the Data

In [19]:
import pandas as pd

# Load supplier datasets
df_supplier1 = pd.read_excel("supplier_data1.xlsx")
df_supplier2 = pd.read_excel("supplier_data2.xlsx")

# Load buyer preferences dataset
df_buyers = pd.read_excel("buyer_preferences.xlsx")

# Display first few rows
print(df_supplier1.head(), df_supplier2.head(), df_buyers.head())

  Quality/Choice  Grade               Finish  Thickness (mm)  Width (mm)  \
0            3rd  C200S  gebeizt und geglüht            2.77        1100   
1            3rd  C300S            ungebeizt            2.65        1075   
2            3rd  C100S  gebeizt und geglüht            2.20        1100   
3            2nd  C100S              gebeizt            2.86        1100   
4            1st  C300S            ungebeizt            2.88        1050   

                          Description  Gross weight (kg)   RP02      RM  \
0                Längs- oder Querisse              13983  333.6   606.2   
1                Längs- oder Querisse              13047  717.7     0.0   
2       Kantenfehler - FS-Kantenrisse              14155  368.9     0.0   
3                Längs- oder Querisse              11381  368.9   601.7   
4  Sollmasse (Gewicht) unterschritten              10072    0.0  1213.0   

   Quantity     AG      AI  
0      0.00  16.11  0.0054  
1      0.00  16.11  0.0046  
2    

In [30]:
print(df_supplier1.columns, df_supplier2.columns, df_buyers.columns)

Index(['Quality/Choice', 'Grade', 'Finish', 'Thickness (mm)', 'Width (mm)',
       'Description', 'Gross weight (kg)', 'RP02', 'RM', 'Quantity', 'AG',
       'AI'],
      dtype='object') Index(['Material', 'Description', 'Article ID', 'Weight (kg)', 'Quantity',
       'Reserved'],
      dtype='object') Index(['Buyer ID', 'Preferred Grade', 'Preferred Finish',
       'Preferred Thickness (mm)', 'Preferred Width (mm)', 'Max Weight (kg)',
       'Min Quantity'],
      dtype='object')


In [23]:
import sqlite3

# Connect to SQLite (or create a new database)
conn = sqlite3.connect("database.db")
cursor = conn.cursor()


# Store data in SQLite tables
df_supplier1.to_sql("supplier1", conn, if_exists="replace", index=False)
df_supplier2.to_sql("supplier2", conn, if_exists="replace", index=False)
df_buyers.to_sql("buyers", conn, if_exists="replace", index=False)

print("Data loaded into SQLite!")

Data loaded into SQLite!


### 2. Standardize Supplier Data

In [25]:
cursor.execute("DROP TABLE IF EXISTS suppliers;")

cursor.execute("""
    CREATE TABLE suppliers AS 
    SELECT 
        "Quality/Choice" AS quality,
        Grade AS grade,
        Finish AS finish,
        "Thickness (mm)" AS thickness,
        "Width (mm)" AS width,
        "Gross weight (kg)" AS weight,
        Quantity AS quantity,
        Description AS description
    FROM supplier1

    UNION ALL

    SELECT 
        NULL AS quality,  -- supplier2 has no quality column
        Material AS grade,
        NULL AS finish,  -- supplier2 has no finish column
        NULL AS thickness,
        NULL AS width,
        "Weight (kg)" AS weight,
        Quantity AS quantity,
        Description AS description
    FROM supplier2;
""")

conn.commit()  # Save changes
print("Supplier table created successfully!")

Supplier table created successfully!


### 3. Clean Missing Values
##### Delete rows where critical fields (grade, thickness, width, finish) are NULL.

In [26]:
cursor.execute("""
    DELETE FROM suppliers 
    WHERE grade IS NULL OR thickness IS NULL OR width IS NULL;
""")
conn.commit()

In [32]:
cursor.execute("PRAGMA table_info(suppliers);")
columns = cursor.fetchall()
print(columns)

[(0, 'quality', 'TEXT', 0, None, 0), (1, 'grade', 'TEXT', 0, None, 0), (2, 'finish', 'TEXT', 0, None, 0), (3, 'thickness', 'REAL', 0, None, 0), (4, 'width', 'INT', 0, None, 0), (5, 'weight', 'INT', 0, None, 0), (6, 'quantity', 'REAL', 0, None, 0), (7, 'description', 'TEXT', 0, None, 0)]


### 4. Match Buyers with Suppliers

In [34]:
cursor.execute("""
    CREATE TABLE recommendations AS
    SELECT 
        b."Buyer ID" AS buyer_id,
        b."Preferred Grade" AS grade,
        b."Preferred Finish" AS finish,
        b."Preferred Thickness (mm)" AS thickness,
        b."Preferred Width (mm)" AS width,
        b."Max Weight (kg)" AS max_weight_kg,
        b."Min Quantity" AS min_quantity,
        s.weight,  -- Correct column name for weight
        s.quantity,
        s.description
    FROM buyers AS b
    JOIN suppliers AS s
    ON b."Preferred Grade" = s.grade
    AND (b."Preferred Finish" = s.finish OR s.finish IS NULL)
    AND (b."Preferred Thickness (mm)" = s.thickness OR s.thickness IS NULL)
    AND (b."Preferred Width (mm)" = s.width OR s.width IS NULL)
    WHERE s.weight <= b."Max Weight (kg)"
    AND s.quantity >= b."Min Quantity";
""")
conn.commit()

### 5. Export Results to CSV

In [35]:
# Step 1: Execute SELECT query
cursor.execute("SELECT * FROM recommendations;")

# Step 2: Fetch all results
rows = cursor.fetchall()

# Step 3: Convert the result into a pandas DataFrame
columns = [column[0] for column in cursor.description]  # Get column names
df_recommendations = pd.DataFrame(rows, columns=columns)

# Step 4: Save the DataFrame to a CSV file
df_recommendations.to_csv("recommendation_table.csv", index=False)
print("Recommendation table saved as CSV!")

Recommendation table saved as CSV!


In [37]:
df_recommendations.head()

,buyer_id,grade,finish,thickness,width,max_weight_kg,min_quantity,weight,quantity,description
